# Experiment 05-04

In [1]:
import sys
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information
import logging

## Configure Logging

In [2]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("ex05_04.log"),
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2022-07-12 02:22:11 INFO     Started


## Show the available Platforms

In [3]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: AMD Accelerated Parallel Processing
Platform 0 - Vendor: Advanced Micro Devices, Inc.
Platform 0 - Version: OpenCL 2.1 AMD-APP (3354.13)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: gfx1010:xnack-
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1675 Mhz
 Device - Compute Units: 18
 Device - Local Memory: 64 KB
 Device - Constant Memory: 6893568 KB
 Device - Global Memory: 8 GB
 Device - Max Buffer/Image Size: 6732 MB
 Device - Max Work Group Size: 256


 --------------------------------------------------------
 Device - Name: gfx1010:xnack-
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1815 Mhz
 Device - Compute Units: 20
 Device - Local Memory: 64 KB
 Device - Constant Memory: 6893568 KB
 Device - Global Memory: 8 GB
 Device - Max Buffer/Image Size: 6732 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [4]:
platform_number = 0
device_number = 1

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [5]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [6]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: get_random_numbers;get_random_string;get_single_hash_nonce;get_single_hash;hash_main;mine_nonce


C:\Users\johnny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pyopencl\__init__.py:274: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


## Mine Nonce

In [7]:
max_nonce = 32

In [8]:
def display_output(zeros_found : dict, nonce : np.ndarray,
                   nonce_len : np.ndarray) -> None:
    """
    Display the output generated by the OpenCL Kernel.
    """
    for i in range(0, 64):
        if nonce_len[i] > 0:
            if (i not in zeros_found):
                nonce_str = nonce[i * max_nonce:i * max_nonce + nonce_len[i]].tobytes().decode('UTF-8')
                zeros_found[i] = nonce_str
                hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))

                logging.info("%4d: [%2d] %32s %64s" % (i, nonce_len[i], nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

In [9]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
window_size = np.uint32(5000000)
nonce = np.zeros(shape=max_nonce * 64, dtype=np.uint8)
nonce_len = np.zeros(shape=64, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

zeros_found = {}

keep_running = True
while (keep_running):
    seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_nonce(
        cl_queue, (256,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len)

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

    # display the results
    display_output(zeros_found, nonce, nonce_len)

    keep_running = True

2022-07-12 02:22:19 INFO     build program: kernel 'mine_nonce' was part of a lengthy source build resulting from a binary cache miss (6.76 s)
2022-07-12 02:24:18 INFO        1: [27]      {y+3$v8 &mjDwPV.V #IV<u>Zlg 08c1970f5de0d80404f4804ffeba0c6a53cbdfb553ed0aef0757dc8510d0dca5
2022-07-12 02:24:18 INFO        2: [29]    MMMMMMMMMMMMMMMMMMMMMMMMMMMMM 0011d8caf17bcbd55a1f50bb185738b4af81c5ec67531fce49359e518e40c100
2022-07-12 02:24:18 INFO        3: [ 5]                            &:^Kh 00036b3455998deec4dc33fd2a10d492c72fe4fb9385df616b8567a9f83760fc
2022-07-12 02:24:18 INFO        4: [ 3]                              =ko 00007c8fcca8b81322aa0f282065e235427d528b7c0e4dca29c2dc84cd1dee5d
2022-07-12 02:24:18 INFO        5: [22]           M,-g)kLTw+9\^B}GnC)c"> 000004a523c06c9497611a9df2213bab8deeea646d0982c6e420ac828ea4cb19
2022-07-12 02:24:18 INFO        6: [23]          ~Yh)G~"Bs-nbPI}y$VY3XQG 000000d451ab093c0a2e0b1098b3ba4fd19867b4daaa347fd8687e3cea025774
2022-07-12 02:24:18 INFO     